### Example: Using BioSet2Vec to compute TFIDF with k_min to k_max on sets

In [1]:
from bioset2vec import BioSet2Vec
import time, json,re
def clean_folder_name(folder_name):
    return re.sub(r'^[^a-zA-Z0-9_-]+|[^a-zA-Z0-9_-]+$', '', folder_name.strip())

In [ ]:
input_file = "input.json"
params = BioSet2Vec.read_parameters_from_file(input_file)
start_time = time.time()
BioSet2Vec.compute(params)
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60


n_core = params.get("n_core")
ram = params.get("ram")
input_folder = params.get('folder_path').split(',')
data = {'execution_time_minutes': execution_time_minutes, "n_core": params.get("n_core"), "input_file": input_folder, "ram": str(ram)}

f_name = clean_folder_name(input_folder[0])

with open("execution_time_core_"+ str(n_core) +"_"+ str(f_name) +".json", 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"Execution time saved in the file: {execution_time_minutes:.4f} min")


24/10/30 16:31:16 WARN Utils: Your hostname, Air-di-Ylenia.homenet.telecomitalia.it resolves to a loopback address: 127.0.0.1; using 192.168.1.7 instead (on interface en0)
24/10/30 16:31:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/10/30 16:31:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/30 16:31:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/Users/yleniagalluzzo/Esperimenti_tfidf_14_07_23/script_app/BioSet2Vec/venv_bioset2vec/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
  0%|          | 0/2 [00:00<?, ?it/s]

hello


(
Numero righe: ,6)size doc (simple count)  = 12
size doc (substring between rows count)= 7
Final size doc (total) = 19


 50%|█████     | 1/2 [00:28<00:28, 28.94s/it]                                   

hello
(
Numero righe: ,6)size doc (simple count)  = 8
size doc (substring between rows count)= 9
Final size doc (total) = 17


...

hello
(
Numero righe: ,7)size doc (simple count)  = 15
size doc (substring between rows count)= 9
Final size doc (total) = 24


 50%|█████     | 1/2 [00:06<00:06,  6.03s/it]                                   

hello
(
Numero righe: ,7)size doc (simple count)  = 10
size doc (substring between rows count)= 12
Final size doc (total) = 22


100%|██████████| 2/2 [00:09<00:00,  4.75s/it]


Execution time saved in the file: 1.5094 min


### Example: BioSet2Vec to Perform only TF on sets with distinct k

In [30]:
from bioset2vec import BioSet2Vec
import time, json,re, os, sys
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext

In [31]:
input_file = "input.json"
params = BioSet2Vec.read_parameters_from_file(input_file)

folder_paths = params.get('folder_path').split(',') # split  quando ho piu set
k_min = params.get('k_min')
k_max = params.get('k_max')
n = params.get('n')
n_core = params.get("n_core")
memory = params.get("ram")
setting_memory = str(memory) + "g"
offHeap_size = params.get("offHeap_size")
path_jar = params.get("path_jar")

if not all(os.path.isdir(folder_path) for folder_path in folder_paths):
    print("Error: One or more specified folder paths are invalid.")
    sys.exit(1)

spark = SparkSession \
            .builder \
            .appName("scala_pyspark") \
            .config("spark.jars", path_jar)\
            .config("spark.driver.cores", str(n_core))\
            .config("spark.driver.memory", setting_memory)\
            .config("spark.executor.cores", str(n_core))\
            .config("spark.rdd.compress", "true")\
            .getOrCreate()
            #.config("spark.memory.offHeap.enabled", "true")\
            #.config("spark.memory.offHeap.size", offHeap_size)\ 
            #.config("spark.cleaner.referenceTracking.cleanCheckpoints", "true")\
            #.config("spark.cleaner.referenceTracking.blocking.shuffle", "true")\
            

sc = spark.sparkContext
sc.setLogLevel("ERROR")
#print("Spark UI running on:", spark.conf.get("spark.ui.port"))
sqlContext = SQLContext(spark)

for folder_path in folder_paths: 
    files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    files = [file for file in files if not file.endswith('.DS_Store')]
    #all files (DNA sequence) in a folder 
    # Example:['./seq_synthetic/seq_synthetic_n1.fa',
    # './seq_synthetic/seq_synthetic_n3.fa',
    # './seq_synthetic/seq_synthetic_n2.fa']
    for file_fasta in files:
        BioSet2Vec.compute_simple_tf(folder_path, sc,sqlContext, file_fasta, k_min, k_max)
        
spark.stop()

  0%|          | 0/2 [00:00<?, ?it/s]

hello
(
Numero righe: ,6)size doc (simple count)  = 12
size doc (substring between rows count)= 7
Final size doc (total) = 19


 50%|█████     | 1/2 [00:20<00:20, 20.73s/it]                                   

hello
(
Numero righe: ,6)size doc (simple count)  = 8
size doc (substring between rows count)= 9
Final size doc (total) = 17


  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

hello
(
Numero righe: ,7)size doc (simple count)  = 15
size doc (substring between rows count)= 9
Final size doc (total) = 24


 50%|█████     | 1/2 [00:02<00:02,  2.38s/it]

hello
(
Numero righe: ,7)size doc (simple count)  = 10
size doc (substring between rows count)= 12
Final size doc (total) = 22


100%|██████████| 2/2 [00:04<00:00,  2.29s/it]


### Use of BioSet2Vec to Perform only TFIDF on sets (we MUST calculate previously of course TF for our sets)

In [ ]:
from bioset2vec import BioSet2Vec
import time, json,re, os, sys
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext

input_file = "input.json"
params = BioSet2Vec.read_parameters_from_file(input_file)

folder_paths = params.get('folder_path').split(',') # split  quando ho piu set
k_min = params.get('k_min')
k_max = params.get('k_max')
n = params.get('n')
n_core = params.get("n_core")
memory = params.get("ram")
setting_memory = str(memory) + "g"
offHeap_size = params.get("offHeap_size")
path_jar = params.get("path_jar")

if not all(os.path.isdir(folder_path) for folder_path in folder_paths):
    print("Error: One or more specified folder paths are invalid.")
    sys.exit(1)
spark = SparkSession \
            .builder \
            .appName("scala_pyspark") \
            .config("spark.jars", path_jar)\
            .config("spark.driver.cores", str(n_core))\
            .config("spark.driver.memory", setting_memory)\
            .config("spark.executor.cores", str(n_core))\
            .config("spark.rdd.compress", "true")\
            .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")
sqlContext = SQLContext(spark)

#For example, we only want to calculate TF-IDF on 3-mers between sets.
k = 3  
pathTF = folder_paths[0] + "/res/resultsTF/tf_" + str(k)
pathTFIDF =  folder_paths[0] + "res/" + "resultsTFIDF/"
tfidf = BioSet2Vec.compute_tfidf(sc, sqlContext, spark, pathTF,pathTFIDF)


In [33]:
tfidf.show()

+-----+-----+---------+--------------------+-------+-----------+---+------------------+--------------------+
|kmers|count|kmers_doc|                  TF|id_file|countShared|  N|               IDF|              TF-IDF|
+-----+-----+---------+--------------------+-------+-----------+---+------------------+--------------------+
|  aaa|    8|       16|                 0.5|set3.fa|          1|  3|1.0986122886681096|  0.5493061443340548|
|  ttt|    4|       16|                0.25|set2.fa|          1|  3|1.0986122886681096|  0.2746530721670274|
|  ggg|    3|       21| 0.14285714285714285|set1.fa|          1|  3|1.0986122886681096|  0.1569446126668728|
|  ccc|    3|       21| 0.14285714285714285|set1.fa|          1|  3|1.0986122886681096|  0.1569446126668728|
|  acg|    1|       16|              0.0625|set2.fa|          1|  3|1.0986122886681096| 0.06866326804175685|
|  cgc|    1|       16|              0.0625|set2.fa|          1|  3|1.0986122886681096| 0.06866326804175685|
|  tta|    1|      

In [ ]:
spark.stop()

### Example: Run test montecarlo 



In [18]:
from bioset2vec import BioSet2Vec
import time, json, re, sys,os
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext


In [ ]:
input_file = "input_montecarlo.json" 
params = BioSet2Vec.read_parameters_from_file(input_file)
BioSet2Vec.compute(params)

In [20]:
input_file = "input_montecarlo.json" 
params = BioSet2Vec.read_parameters_from_file(input_file)

folder_paths = params.get('folder_path').split(',') # split  quando ho piu set
k_min = params.get('k_min')
k_max = params.get('k_max')
n = params.get('n')
n_core = params.get("n_core")
memory = params.get("ram")
setting_memory = str(memory) + "g"
offHeap_size = params.get("offHeap_size")
path_jar = params.get("path_jar")


if not all(os.path.isdir(folder_path) for folder_path in folder_paths):
    print("Error: One or more specified folder paths are invalid.")
    sys.exit(1)

spark = SparkSession \
            .builder \
            .appName("scala_pyspark") \
            .config("spark.jars", path_jar)\
            .config("spark.driver.cores", str(n_core))\
            .config("spark.driver.memory", setting_memory)\
            .config("spark.executor.cores", str(n_core))\
            .config("spark.rdd.compress", "true")\
            .getOrCreate()

#sc = spark.sparkContext
#sc.setLogLevel("ERROR")
#sqlContext = SQLContext(spark)


start_time = time.time()

BioSet2Vec.get_result_montecarlo(spark, input_file)

end_time = time.time()


execution_time_minutes = (end_time - start_time) / 60
print(f"Execution time: {execution_time_minutes:.4f} min")



{'path_jar': './bioset2vec/bioft.jar', 'folder_path': './set_synthetic/', 'k_min': 3, 'k_max': 4, 'n': 5, 'n_core': 6, 'ram': 6, 'offHeap_size': 2}


Execution time: 0.2538 min


In [21]:
import pandas as pd

check_df_score_montecarlo= pd.read_parquet(folder_paths[0] + "res/montecarlo_score/")
for set in check_df_score_montecarlo.id_file.unique().tolist():
    score_montecarlo_set = pd.read_parquet(folder_paths[0] + "res/montecarlo_score/result_"+set)
    display(score_montecarlo_set)

,kmers,max_TF-IDF,count,id_file,kmers_doc,TF_old,kmers_doc_union,TF,countShared,N,IDF,TF-IDF_real,is_score_valid
0,ccc,0.073241,3,set1.fa,21,0.142857,40,0.075,1,3,1.098612,0.082396,yes
1,ggg,0.027465,3,set1.fa,21,0.142857,40,0.075,1,3,1.098612,0.082396,yes
2,cccc,0.036620,2,set1.fa,19,0.105263,40,0.050,1,3,1.098612,0.054931,yes
3,gggg,0.000000,2,set1.fa,19,0.105263,40,0.050,1,3,1.098612,0.054931,yes
4,accc,0.036620,1,set1.fa,19,0.052632,40,0.025,1,3,1.098612,0.027465,no
5,atac,0.027465,1,set1.fa,19,0.052632,40,0.025,1,3,1.098612,0.027465,yes
6,cccg,0.036620,1,set1.fa,19,0.052632,40,0.025,1,3,1.098612,0.027465,no
7,ccg,0.036620,1,set1.fa,21,0.047619,40,0.025,1,3,1.098612,0.027465,no
8,ccgg,0.036620,1,set1.fa,19,0.052632,40,0.025,1,3,1.098612,0.027465,no
9,cgg,0.036620,1,set1.fa,21,0.047619,40,0.025,1,3,1.098612,0.027465,no


,kmers,max_TF-IDF,count,id_file,kmers_doc,TF_old,kmers_doc_union,TF,countShared,N,IDF,TF-IDF_real,is_score_valid
0,ttt,0.073241,4,set2.fa,16,0.250000,30,0.133333,1,3,1.098612,0.146482,yes
1,tttt,0.036620,3,set2.fa,14,0.214286,30,0.100000,1,3,1.098612,0.109861,yes
2,acg,0.027465,1,set2.fa,16,0.062500,30,0.033333,1,3,1.098612,0.036620,yes
3,acgc,0.027465,1,set2.fa,14,0.071429,30,0.033333,1,3,1.098612,0.036620,yes
4,atat,0.000000,1,set2.fa,14,0.071429,30,0.033333,1,3,1.098612,0.036620,yes
5,cga,0.036620,1,set2.fa,16,0.062500,30,0.033333,1,3,1.098612,0.036620,yes
6,cgc,0.036620,1,set2.fa,16,0.062500,30,0.033333,1,3,1.098612,0.036620,yes
7,cgcg,0.036620,1,set2.fa,14,0.071429,30,0.033333,1,3,1.098612,0.036620,yes
8,ctac,0.000000,1,set2.fa,14,0.071429,30,0.033333,1,3,1.098612,0.036620,yes
9,gcg,0.036620,1,set2.fa,16,0.062500,30,0.033333,1,3,1.098612,0.036620,yes


,kmers,max_TF-IDF,count,id_file,kmers_doc,TF_old,kmers_doc_union,TF,countShared,N,IDF,TF-IDF_real,is_score_valid
0,aaa,0.054931,8,set3.fa,16,0.500000,30,0.266667,1,3,1.098612,0.292963,yes
1,aaaa,0.027465,7,set3.fa,14,0.500000,30,0.233333,1,3,1.098612,0.256343,yes
2,aac,0.054931,1,set3.fa,16,0.062500,30,0.033333,2,3,0.405465,0.013516,no
3,aacc,0.036620,1,set3.fa,14,0.071429,30,0.033333,2,3,0.405465,0.013516,no
4,ctaa,0.013516,1,set3.fa,14,0.071429,30,0.033333,2,3,0.405465,0.013516,yes
5,taa,0.027465,1,set3.fa,16,0.062500,30,0.033333,2,3,0.405465,0.013516,no
6,taac,0.027031,1,set3.fa,14,0.071429,30,0.033333,2,3,0.405465,0.013516,no
7,acc,0.013516,2,set3.fa,16,0.125000,30,0.066667,3,3,0.000000,0.000000,no
8,acct,0.027465,2,set3.fa,14,0.142857,30,0.066667,3,3,0.000000,0.000000,no
9,cct,0.073241,2,set3.fa,16,0.125000,30,0.066667,3,3,0.000000,0.000000,no


In [22]:
spark.stop()